## Import tools

In [4]:
import requests
from bs4 import BeautifulSoup as bs
import re
import time, random
import pandas as pd

## Check if scraping allowed with robots.txt

In [5]:
url_robots = 'https://www.boxofficemojo.com/'
response_robots = requests.get(url_robots)
#print(response_robots.text)

In [6]:
## Try to scrape a single movie
#url = 'https://www.boxofficemojo.com/release/rl1182631425/?ref_=bo_yld_table_1'
#response = requests.get(url)
#response.status_code

In [7]:
## put into BeautifulSoup
#page_html = response.text
#soup = bs(page_html)
#type(soup)

### Define function to extract title of movie to use as key in data dictionary

In [14]:
#def get_one_movie_name(soup):
 #   name = soup.find('h1').text
  #  return name
    

### Define function to extract movie features

In [174]:
def get_one_movie_data(soup):
    data = {}
    ## Get name
    name = soup.find('h1').text
    data['Name'] = name
    ## Get data from first table
    grosses = soup.find(class_ = 'a-section a-spacing-none mojo-performance-summary-table')
    #kinds = []
    #for item in grosses.find_all(class_ = 'a-size-small'):
    #    raw = str.strip(item.text)
    #    clean = re.sub('\(.*\)', '', raw)
    #    kinds.append(str.strip(clean))
    money = []
    for item in grosses.find_all(class_ = 'money'):
        money.append(str.strip(item.text))

    ## Put them in a dictionary!
   # gross_data = {kinds[i]: money[i] for i in range(len(kinds))}
    #gross_data
    
    ## Put in data dictionary that will be returned
    try:
        data['Domestic'] = money[0]
    except:
        data['Domestic'] = 'Null'
    try:
        data['International'] = money[1]
    except:
        data['International'] = 'Null'
    try: 
        data['Worldwide'] = money[2]
    except:
        data['Worldwide'] = "Null"
   # data['Domestic'] = gross_data['Domestic']
   # data['International'] = gross_data['International']
   # data['Worldwide'] = gross_data['Worldwide']
    
    ## Get data from second table and add to data dictionary that will be returned
    table_data = soup.find(class_ = 'a-section a-spacing-none mojo-summary-values mojo-hidden-from-mobile')
    for item in table_data.find_all(class_ = "a-section a-spacing-none"):
        if str.strip(item.find('span').text) == 'Distributor':
            raw = (str.strip(item.find('span').next.next.text))
            cleaned = re.sub('See full company information', '', raw)
            data['Distributor'] = cleaned
        elif str.strip(item.find('span').text) == 'Opening':
            value = str.strip(item.find(class_ = 'money').text)
            data['Opening'] = value
        elif str.strip(item.find('span').text) == 'Budget':
            value = str.strip(item.find(class_ = 'money').text)
            data['Budget'] = value
        elif re.match('Release Date', str.strip(item.find('span').text)):
            date = (str.strip(item.find('span').next.next.text))
            if re.match('.*\(', date):
                d_list = re.split('\(', date)
            else:
                d_list = date.split('\n')
            data['Release Date'] = str.strip(d_list[0])
        elif str.strip(item.find('span').text) == 'MPAA':
            rating = (str.strip(item.find('span').next.next.text))
            data['MPAA'] = rating
        elif str.strip(item.find('span').text) == 'Running Time':
            time = (str.strip(item.find('span').next.next.text))
            data['Running Time'] = time
        elif str.strip(item.find('span').text) == 'Genres':
            genres = (str.strip(item.find('span').next.next.text))
            g_list = genres.split('\n')
            final_list = []
            for item in g_list:
                item = str.strip(item)
                if len(item) > 1:
                    final_list.append(item)
            data['Genres'] = final_list
        elif str.strip(item.find('span').text) == 'In Release':
            release = (str.strip(item.find('span').next.next.text))
            release_weeks = re.sub('\/.*weeks', '', release)
            data['In Release'] = str.strip(release_weeks)
        elif str.strip(item.find('span').text) == 'Widest Release':
            w_release = (str.strip(item.find('span').next.next.text))
            data['Widest Release'] = str.strip(w_release)
        else:
            pass

    return data

### Define function that takes a list of URLs as input and returns a list of dictionaries with the fields as keys and the movie features as the values

In [62]:
def get_many_movies(url_list):
    all_data = []
    for item in url_list:
        time.sleep(2)
        response = requests.get(item)
        page_html = response.text
        soup = bs(page_html)
        all_data.append(get_one_movie_data(soup))
    return all_data

In [22]:
## See if it works on three test urls
test_list = ['https://www.boxofficemojo.com/release/rl1182631425/?ref_=bo_yld_table_1', 'https://www.boxofficemojo.com/release/rl3490022913/?ref_=bo_yld_table_2', 'https://www.boxofficemojo.com/release/rl1908310529/?ref_=bo_yld_table_3']

test = get_many_movies(test_list)



In [23]:
## See if it will go into a dataframe!!
df = pd.DataFrame(test)
df.head()

,name,Domestic,International,Worldwide,Distributor,Opening,Budget,Release Date,MPAA,Running Time,Genres,In Release,Widest Release
0,Bad Boys for Life,"$206,305,244","$220,200,000","$426,505,244",Sony Pictures Entertainment (SPE),"$62,504,105","$90,000,000","Jan 17, 2020",R,2 hr 4 min,"[Action, Comedy, Crime, Thriller]",350 days,"3,775 theaters"
1,Shang-Chi and the Legend of the Ten Rings,"$224,543,292","$207,689,718","$432,233,010",Walt Disney Studios Motion Pictures,"$75,388,688",NaN,"Sep 3, 2021",PG-13,2 hr 12 min,"[Action, Adventure, Fantasy, Sci-Fi]",159 days,"4,300 theaters"
2,Venom: Let There Be Carnage,"$213,550,366","$288,500,000","$502,050,366",Sony Pictures Entertainment (SPE),"$90,033,210",NaN,"Oct 1, 2021",PG-13,1 hr 37 min,"[Action, Adventure, Sci-Fi, Thriller]",131 days,"4,225 theaters"


In [47]:
### Scrape URLS
## 2021 index page url
twenty_one_url = 'https://www.boxofficemojo.com/year/2021/?ref_=bo_yl_table_2'
response = requests.get(twenty_one_url)
##response.status_code
## put into BeautifulSoup
page_html = response.text
soup = bs(page_html)
type(soup)

bs4.BeautifulSoup

In [47]:
## Returns URLS from 2021 Domestic Movies page
def get_index_urls(url):
    response = requests.get(url)
    ## put into BeautifulSoup
    page_html = response.text
    soup = bs(page_html)
    cleaned = []
    raw = []
    for link in soup.find('table').find_all(class_ = 'a-link-normal'):
        raw.append(link['href'])
        for item in raw:
            if re.match('\/release\/.*', item):
                if ('https://www.boxofficemojo.com' + item) not in cleaned:
                    cleaned.append('https://www.boxofficemojo.com' + item)
                else:
                    pass
            else:
                pass
    return(cleaned)
    

In [54]:
## Get index of URLs for 2021
twenty_one_urls = get_index_urls('https://www.boxofficemojo.com/year/2021/?ref_=bo_yl_table_2')

In [55]:
len(twenty_one_urls)

200

In [56]:
twenty_one_urls[-20:-1]

['https://www.boxofficemojo.com/release/rl2114487041/?ref_=bo_yld_table_181',
 'https://www.boxofficemojo.com/release/rl2081522433/?ref_=bo_yld_table_182',
 'https://www.boxofficemojo.com/release/rl3077669633/?ref_=bo_yld_table_183',
 'https://www.boxofficemojo.com/release/rl3669066241/?ref_=bo_yld_table_184',
 'https://www.boxofficemojo.com/release/rl1644593921/?ref_=bo_yld_table_185',
 'https://www.boxofficemojo.com/release/rl3512894209/?ref_=bo_yld_table_186',
 'https://www.boxofficemojo.com/release/rl1846182657/?ref_=bo_yld_table_187',
 'https://www.boxofficemojo.com/release/rl760382209/?ref_=bo_yld_table_188',
 'https://www.boxofficemojo.com/release/rl1735361281/?ref_=bo_yld_table_189',
 'https://www.boxofficemojo.com/release/rl3104540161/?ref_=bo_yld_table_190',
 'https://www.boxofficemojo.com/release/rl252543745/?ref_=bo_yld_table_191',
 'https://www.boxofficemojo.com/release/rl2658632449/?ref_=bo_yld_table_192',
 'https://www.boxofficemojo.com/release/rl2332459777/?ref_=bo_yld_

In [57]:
## Get index of URLs for 2020
twenty_two_urls = get_index_urls('https://www.boxofficemojo.com/year/2020/?ref_=bo_yl_table_3')

In [58]:
len(twenty_two_urls)

200

# Attempt to scrape 2021 data!

In [59]:
## Get index of URLs for 2021
twenty_one_urls = get_index_urls('https://www.boxofficemojo.com/year/2021/?ref_=bo_yl_table_2')

In [67]:
## Scrape data from 2021 URLs
twenty_one_urls_part_one = twenty_one_urls[0:99]
twenty_one_data_part_one = get_many_movies(twenty_one_urls_part_one)

In [97]:
## See if it will go into a dataframe!!
df_21 = pd.DataFrame(twenty_one_data_part_one)
df_21.head(5)

,Name,Domestic,International,Worldwide,Distributor,Opening,Release Date,MPAA,Running Time,Genres,In Release,Widest Release,Budget
0,Spider-Man: No Way Home,"$750,429,025","$1,027,000,000","$1,777,429,025",Sony Pictures Entertainment (SPE),"$260,138,569","Dec 17, 2021",PG-13,2 hr 28 min,"[Action, Adventure, Fantasy, Sci-Fi]",54 days,"4,336 theaters",NaN
1,Shang-Chi and the Legend of the Ten Rings,"$224,543,292","$207,689,718","$432,233,010",Walt Disney Studios Motion Pictures,"$75,388,688","Sep 3, 2021",PG-13,2 hr 12 min,"[Action, Adventure, Fantasy, Sci-Fi]",159 days,"4,300 theaters",NaN
2,Venom: Let There Be Carnage,"$213,550,366","$288,500,000","$502,050,366",Sony Pictures Entertainment (SPE),"$90,033,210","Oct 1, 2021",PG-13,1 hr 37 min,"[Action, Adventure, Sci-Fi, Thriller]",131 days,"4,225 theaters",NaN
3,Black Widow,"$183,651,655","$195,979,696","$379,631,351",Walt Disney Studios Motion Pictures,"$80,366,312","Jul 9, 2021",PG-13,2 hr 14 min,"[Action, Adventure, Sci-Fi]",215 days,"4,275 theaters",NaN
4,F9: The Fast Saga,"$173,005,945","$553,223,556","$726,229,501",Universal Pictures,"$70,043,165","Jun 25, 2021",PG-13,2 hr 23 min,"[Action, Crime, Thriller]",229 days,"4,203 theaters",NaN


In [107]:
df_21[pd.isnull(df_21['Release Date'])]

,Name,Domestic,International,Worldwide,Distributor,Opening,Release Date,MPAA,Running Time,Genres,In Release,Widest Release,Budget


In [73]:
## Pickle this first dataframe!
df_21.to_pickle('2021 movie data part 1')

In [74]:
len(twenty_one_urls)

200

In [75]:
twenty_one_urls_part_two = twenty_one_urls[100:199]
twenty_one_data_part_two = get_many_movies(twenty_one_urls_part_two)

In [76]:
df_21_2 = pd.DataFrame(twenty_one_data_part_two)
df_21_2.head(5)

,Name,Domestic,International,Worldwide,Distributor,Opening,Release Date,MPAA,Running Time,Genres,In Release,Widest Release,Budget
0,The Card Counter,"$2,657,850","$1,884,895","$4,542,745",Focus Features,"$1,039,580","Sep 10, 2021",R,1 hr 51 min,"[Crime, Drama, Thriller]",152 days,739 theaters,NaN
1,Land,"$2,577,830","$586,416","$3,164,246",Focus Features,"$899,810","Feb 12, 2021",PG-13,1 hr 29 min,"[Adventure, Drama]",362 days,"1,349 theaters",NaN
2,The War with Grandpa,"$21,277,892","$19,432,044","$40,709,936",101 Studios,"$3,623,880","Oct 9, 2020",PG,1 hr 34 min,"[Comedy, Drama, Family]",488 days,"2,365 theaters",NaN
3,The Eyes of Tammy Faye,"$2,404,127","$5,586","$2,409,713",Searchlight Pictures,"$652,358","Sep 17, 2021",PG-13,2 hr 6 min,"[Biography, Drama, Romance]",145 days,"1,352 theaters",NaN
4,"Summer of Soul (...Or, When the Revolution Cou...","$2,320,649","$1,370,490","$3,691,139",Searchlight Pictures,"$802,054","Jun 25, 2021",PG-13,1 hr 58 min,"[Documentary, Music]",229 days,752 theaters,NaN


In [106]:
df_21_2[pd.isnull(df_21_2['Release Date'])]

,Name,Domestic,International,Worldwide,Distributor,Opening,Release Date,MPAA,Running Time,Genres,In Release,Widest Release,Budget


In [77]:
## Pickle this first dataframe!
df_21_2.to_pickle('2021 movie data part 2')

# Attempt to scrape 2020 data!

In [78]:
## Get index of URLs for 2020
twenty_urls = get_index_urls('https://www.boxofficemojo.com/year/2020/?ref_=bo_yl_table_3')
print(len(twenty_urls))


200


In [175]:
## Scrape data from 2020 URLs
twenty_data = get_many_movies(twenty_urls)

In [176]:
df_20 = pd.DataFrame(twenty_data)
df_20.head(5)

,Name,Domestic,International,Worldwide,Distributor,Opening,Budget,Release Date,MPAA,Running Time,Genres,In Release,Widest Release
0,Bad Boys for Life,"$206,305,244","$220,200,000","$426,505,244",Sony Pictures Entertainment (SPE),"$62,504,105","$90,000,000","Jan 17, 2020",R,2 hr 4 min,"[Action, Comedy, Crime, Thriller]",350 days,"3,775 theaters"
1,1917,"$159,227,644","$225,691,745","$384,919,389",Universal Pictures,"$576,216","$95,000,000","Dec 25, 2019",R,1 hr 59 min,"[Action, Drama, War]",373 days,"3,987 theaters"
2,Sonic the Hedgehog,"$148,974,665","$170,741,018","$319,715,683",Paramount Pictures,"$58,018,348","$85,000,000","Feb 14, 2020",PG,1 hr 39 min,"[Action, Adventure, Comedy, Sci-Fi]",687 days,"4,198 theaters"
3,Jumanji: The Next Level,"$320,314,960","$479,744,747","$800,059,707",Sony Pictures Entertainment (SPE),"$59,251,543","$125,000,000","Dec 13, 2019",PG-13,2 hr 3 min,"[Action, Adventure, Comedy, Fantasy]",385 days,"4,227 theaters"
4,Star Wars: Episode IX - The Rise of Skywalker,"$515,202,542","$558,941,706","$1,074,144,248",Walt Disney Studios Motion Pictures,"$177,383,864","$275,000,000","Dec 20, 2019",PG-13,2 hr 21 min,"[Action, Adventure, Fantasy, Sci-Fi]",378 days,"4,406 theaters"


In [177]:
df_20[pd.isnull(df_20['Release Date'])]

,Name,Domestic,International,Worldwide,Distributor,Opening,Budget,Release Date,MPAA,Running Time,Genres,In Release,Widest Release


In [178]:
## Pickle this dataframe!
df_20.to_pickle('2020 movie data')

# Attempt to scrape 2019 data!

In [85]:
## Get index of URLs for 2019
nineteen_urls = get_index_urls('https://www.boxofficemojo.com/year/2019/?ref_=bo_yl_table_4')
print(len(nineteen_urls))

200


In [179]:
## Scrape data from 2019 URLs
nineteen_data = get_many_movies(nineteen_urls)

In [180]:
df_19 = pd.DataFrame(nineteen_data)
df_19.head(5)

,Name,Domestic,International,Worldwide,Distributor,Opening,Budget,Release Date,MPAA,Running Time,Genres,In Release,Widest Release
0,Avengers: Endgame,"$858,373,000","$1,939,128,328","$2,797,501,328",Walt Disney Studios Motion Pictures,"$357,115,007","$356,000,000","Apr 26, 2019",PG-13,3 hr 1 min,"[Action, Adventure, Drama, Sci-Fi]",250 days,"4,662 theaters"
1,The Lion King,"$543,638,043","$1,113,305,351","$1,656,943,394",Walt Disney Studios Motion Pictures,"$191,770,759","$260,000,000","Jul 19, 2019",PG,1 hr 58 min,"[Adventure, Animation, Drama, Family, Musical]",168 days,"4,802 theaters"
2,Toy Story 4,"$434,038,008","$639,356,585","$1,073,394,593",Walt Disney Studios Motion Pictures,"$120,908,065","$200,000,000","Jun 21, 2019",G,1 hr 40 min,"[Adventure, Animation, Comedy, Family, Fantasy]",196 days,"4,575 theaters"
3,Frozen II,"$477,373,578","$972,653,355","$1,450,026,933",Walt Disney Studios Motion Pictures,"$130,263,358","$150,000,000","Nov 22, 2019",PG,1 hr 43 min,"[Adventure, Animation, Comedy, Family, Fantasy...",406 days,"4,440 theaters"
4,Captain Marvel,"$426,829,839","$701,444,955","$1,128,274,794",Walt Disney Studios Motion Pictures,"$153,433,423","$160,000,000","Mar 8, 2019",PG-13,2 hr 3 min,"[Action, Adventure, Sci-Fi]",299 days,"4,310 theaters"


In [181]:
df_19[pd.isnull(df_19['Release Date'])]

,Name,Domestic,International,Worldwide,Distributor,Opening,Budget,Release Date,MPAA,Running Time,Genres,In Release,Widest Release


In [182]:
## Pickle this dataframe!
df_19.to_pickle('2019 movie data')

# Attempt to scrape 2018 data!

In [91]:
## Get index of URLs for 2018
eighteen_urls = get_index_urls('https://www.boxofficemojo.com/year/2018/?ref_=bo_yl_table_5')
print(len(eighteen_urls))

200


In [183]:
## Scrape data from 2018 URLs
eighteen_data = get_many_movies(eighteen_urls)

In [184]:
df_18 = pd.DataFrame(eighteen_data)
df_18.tail(5)

,Name,Domestic,International,Worldwide,Distributor,Opening,Release Date,MPAA,Running Time,Genres,In Release,Widest Release,Budget
195,The Disaster Artist,"$21,120,616","$8,700,000","$29,820,616",A24,"$1,211,345","Dec 1, 2017",R,1 hr 44 min,"[Biography, Comedy, Drama]",396 days,"1,010 theaters",NaN
196,2001: A Space Odyssey,"$3,236,321","$848,261","$4,084,582",Warner Bros.,"$202,759","May 18, 2018",NaN,2 hr 29 min,"[Adventure, Sci-Fi]",228 days,13 theaters,"$12,000,000"
197,2.0,"$4,152,000","$32,793,969","$36,945,969",,"$2,775,000","Nov 30, 2018",NaN,2 hr 28 min,"[Action, Sci-Fi, Thriller]",35 days,314 theaters,NaN
198,The Leisure Seeker,"$3,226,443","$7,245,737","$10,472,180",Sony Pictures Classics,NaN,"Dec 15, 2017",R,1 hr 52 min,"[Adventure, Comedy, Drama, Romance]",382 days,353 theaters,NaN
199,The Sisters Brothers,"$3,143,056","$10,000,000","$13,143,056",Annapurna Pictures,"$115,575","Sep 21, 2018",R,2 hr 2 min,"[Crime, Drama, Western]",105 days,"1,141 theaters",NaN


In [185]:
df_18[pd.isnull(df_18['Release Date'])]

,Name,Domestic,International,Worldwide,Distributor,Opening,Release Date,MPAA,Running Time,Genres,In Release,Widest Release,Budget


In [186]:
## Pickle this dataframe!
df_18.to_pickle('2018 movie data')

# Attempt to scrape 2017 data!

In [187]:
## Get index of URLs for 2017
seventeen_urls = get_index_urls('https://www.boxofficemojo.com/year/2017/?ref_=bo_yl_table_6')
print(len(seventeen_urls))

200


In [188]:
## Scrape data from 2017 URLs
seventeen_data = get_many_movies(seventeen_urls)

In [189]:
df_17 = pd.DataFrame(seventeen_data)
df_17.head(5)

,Name,Domestic,International,Worldwide,Distributor,Opening,Budget,Release Date,MPAA,Running Time,Genres,In Release,Widest Release
0,Star Wars: Episode VIII - The Last Jedi,"$620,181,382","$712,358,507","$1,332,539,889",Walt Disney Studios Motion Pictures,"$220,009,584","$317,000,000","Dec 15, 2017",PG-13,2 hr 32 min,"[Action, Adventure, Fantasy, Sci-Fi]",382 days,"4,232 theaters"
1,Beauty and the Beast,"$504,014,165","$759,506,961","$1,263,521,126",Walt Disney Studios Motion Pictures,"$174,750,616","$160,000,000","Mar 17, 2017",PG,2 hr 9 min,"[Adventure, Family, Fantasy, Musical, Romance]",290 days,"4,210 theaters"
2,Wonder Woman,"$412,563,408","$409,283,604","$821,847,012",Warner Bros.,"$103,251,471","$149,000,000","Jun 2, 2017",PG-13,2 hr 21 min,"[Action, Adventure, Fantasy, Sci-Fi, War]",217 days,"4,165 theaters"
3,Guardians of the Galaxy Vol. 2,"$389,813,101","$473,942,950","$863,756,051",Walt Disney Studios Motion Pictures,"$146,510,104","$200,000,000","May 5, 2017",PG-13,2 hr 16 min,"[Action, Adventure, Comedy, Sci-Fi]",241 days,"4,347 theaters"
4,Spider-Man: Homecoming,"$334,201,140","$545,965,784","$880,166,924",Sony Pictures Entertainment (SPE),"$117,027,503","$175,000,000","Jul 7, 2017",PG-13,2 hr 13 min,"[Action, Adventure, Sci-Fi]",182 days,"4,348 theaters"


In [190]:
df_17[pd.isnull(df_17['Release Date'])]

,Name,Domestic,International,Worldwide,Distributor,Opening,Budget,Release Date,MPAA,Running Time,Genres,In Release,Widest Release


In [191]:
## Pickle this dataframe!
df_17.to_pickle('2017 movie data')

# Attempt to scrape 2016 data!

In [192]:
## Get index of URLs for 2016
sixteen_urls = get_index_urls('https://www.boxofficemojo.com/year/2016/?ref_=bo_yl_table_7')
print(len(sixteen_urls))

200


In [193]:
## Scrape data from 2016 URLs
sixteen_data = get_many_movies(sixteen_urls)

In [194]:
df_16 = pd.DataFrame(sixteen_data)
df_16.head(5)

,Name,Domestic,International,Worldwide,Distributor,Opening,Release Date,MPAA,Running Time,Genres,In Release,Widest Release,Budget
0,Finding Dory,"$486,295,561","$542,275,328","$1,028,570,889",Walt Disney Studios Motion Pictures,"$135,060,273","Jun 17, 2016",PG,1 hr 37 min,"[Adventure, Animation, Comedy, Family]",203 days,"4,305 theaters",NaN
1,Rogue One: A Star Wars Story,"$532,177,324","$523,879,949","$1,056,057,273",Walt Disney Studios Motion Pictures,"$155,081,681","Dec 16, 2016",PG-13,2 hr 13 min,"[Action, Adventure, Sci-Fi]",381 days,"4,157 theaters","$200,000,000"
2,Captain America: Civil War,"$408,084,349","$745,211,944","$1,153,296,293",Walt Disney Studios Motion Pictures,"$179,139,142","May 6, 2016",PG-13,2 hr 27 min,"[Action, Adventure, Sci-Fi]",240 days,"4,226 theaters","$250,000,000"
3,The Secret Life of Pets,"$368,384,330","$507,073,607","$875,457,937",Universal Pictures,"$104,352,905","Jul 8, 2016",PG,1 hr 27 min,"[Adventure, Animation, Comedy, Family]",182 days,"4,381 theaters","$75,000,000"
4,The Jungle Book,"$364,001,123","$602,549,477","$966,550,600",Walt Disney Studios Motion Pictures,"$103,261,464","Apr 15, 2016",PG,1 hr 46 min,"[Adventure, Drama, Family, Fantasy]",261 days,"4,144 theaters","$175,000,000"


In [195]:
df_16[pd.isnull(df_16['Release Date'])]

,Name,Domestic,International,Worldwide,Distributor,Opening,Release Date,MPAA,Running Time,Genres,In Release,Widest Release,Budget


In [196]:
## Pickle this dataframe!
df_16.to_pickle('2016 movie data')

## Cleaning

In [116]:
## Try to scrape a single movie
url = 'https://www.boxofficemojo.com/release/rl2969994753/?ref_=bo_yld_table_2'
response = requests.get(url)
#response.status_code

## put into BeautifulSoup
page_html = response.text
soup_test = bs(page_html)
type(soup_test)

get_one_movie_data(soup_test)




{'Name': '1917',
 'Domestic': '$159,227,644',
 'International': '$225,691,745',
 'Worldwide': '$384,919,389',
 'Distributor': 'Universal Pictures',
 'Opening': '$576,216',
 'Budget': '$95,000,000',
 'MPAA': 'R',
 'Running Time': '1 hr 59 min',
 'Genres': ['Action', 'Drama', 'War'],
 'In Release': '373 days',
 'Widest Release': '3,987 theaters'}

In [167]:
def test_date(soup):
    data = {}
    ## Get name
    name = soup.find('h1').text
    data['Name'] = name
    ## Get data from first table
    grosses = soup.find(class_ = 'a-section a-spacing-none mojo-performance-summary-table')
    #kinds = []
    #for item in grosses.find_all(class_ = 'a-size-small'):
    #    raw = str.strip(item.text)
    #    clean = re.sub('\(.*\)', '', raw)
    #    kinds.append(str.strip(clean))
    money = []
    for item in grosses.find_all(class_ = 'money'):
        money.append(str.strip(item.text))

    ## Put them in a dictionary!
   # gross_data = {kinds[i]: money[i] for i in range(len(kinds))}
    #gross_data
    
    ## Put in data dictionary that will be returned
    try:
        data['Domestic'] = money[0]
    except:
        data['Domestic'] = 'Null'
    try:
        data['International'] = money[1]
    except:
        data['International'] = 'Null'
    try: 
        data['Worldwide'] = money[2]
    except:
        data['Worldwide'] = "Null"
   # data['Domestic'] = gross_data['Domestic']
   # data['International'] = gross_data['International']
   # data['Worldwide'] = gross_data['Worldwide']
    
    ## Get data from second table and add to data dictionary that will be returned
    table_data = soup.find(class_ = 'a-section a-spacing-none mojo-summary-values mojo-hidden-from-mobile')
    for item in table_data.find_all(class_ = "a-section a-spacing-none"):
     #   print(item)
      #  if str.strip(item.find('span').text) == 'Distributor':
        #    raw = (str.strip(item.find('span').next.next.text))
        #    cleaned = re.sub('See full company information', '', raw)
        #    data['Distributor'] = cleaned
       # elif str.strip(item.find('span').text) == 'Opening':
       #     value = str.strip(item.find(class_ = 'money').text)
      #      data['Opening'] = value
      #  elif str.strip(item.find('span').text) == 'Budget':
       #     value = str.strip(item.find(class_ = 'money').text)
       #     data['Budget'] = value
       # if str.strip(item.find('span').text) == 'Release Date':
       #     print('Success')
        #    date = (str.strip(item.find('span').next.next.text))
        #    d_list = date.split('\n')
        #    print(d_list)
        #    d_list_2 = d_list.split('\(')
         #   print(d_list_2)
         #   data['Release Date'] = str.strip(d_list_2[0])
        if re.match('Release Date', str.strip(item.find('span').text)):
            date = (str.strip(item.find('span').next.next.text))
            if re.match('.*\(', date):
                d_list = re.split('\(', date)
            else:
                d_list = date.split('\n')
            data['Release Date'] = str.strip(d_list[0])
       # elif str.strip(item.find('span').text) == 'MPAA':
        #    rating = (str.strip(item.find('span').next.next.text))
       #     data['MPAA'] = rating
       # elif str.strip(item.find('span').text) == 'Running Time':
         #   time = (str.strip(item.find('span').next.next.text))
        #    data['Running Time'] = time
       # elif str.strip(item.find('span').text) == 'Genres':
       #     genres = (str.strip(item.find('span').next.next.text))
       #     g_list = genres.split('\n')
       #     final_list = []
       #     for item in g_list:
       #         item = str.strip(item)
       #         if len(item) > 1:
       #             final_list.append(item)
       #     data['Genres'] = final_list
       # elif str.strip(item.find('span').text) == 'In Release':
       #     release = (str.strip(item.find('span').next.next.text))
       #     release_weeks = re.sub('\/.*weeks', '', release)
       #     data['In Release'] = str.strip(release_weeks)
       # elif str.strip(item.find('span').text) == 'Widest Release':
       #     w_release = (str.strip(item.find('span').next.next.text))
        #    data['Widest Release'] = str.strip(w_release)
        else:
            pass
    print(data)

#    return data

In [173]:
test_date(soup_test)

{'Name': '1917', 'Domestic': '$159,227,644', 'International': '$225,691,745', 'Worldwide': '$384,919,389', 'Release Date': 'Dec 25, 2019'}


In [171]:
## Try to scrape a single movie test
url_test = 'https://www.boxofficemojo.com/release/rl1182631425/?ref_=bo_yld_table_1'
response_test = requests.get(url_test)
response_test.status_code

200

In [172]:
## put into BeautifulSoup
page_html_test = response_test.text
soup_test_2 = bs(page_html_test)
test_date(soup_test_2)

{'Name': 'Bad Boys for Life', 'Domestic': '$206,305,244', 'International': '$220,200,000', 'Worldwide': '$426,505,244', 'Release Date': 'Jan 17, 2020'}


## Try to merge data

In [ ]:
twenty_one_data_part_one

In [ ]:
## Syntax to save
## unemp_data.loc[:, 'UNRATENSA'] = unemp_data['UNRATENSA'].astype(float)